## **Data Science Aplicado a las Finanzas** 🚀
### **Sesión 6: Modelos de Regresión**

Andrés C. Medina Sanhueza

Senior Datascientist Engineer 

anmedinas@gmail.com

In [ ]:
from matplotlib import pyplot as plt 
import statsmodels.api as sm
import pandas_datareader.data as web 
import pandas as pd 
import seaborn as sns
import yfinance as yf
import numpy as np

sns.set_style("dark")

## 🐝 ¿Qué es un modelo de regresión?

Un modelo de regresión consiste en un modelo econometrico nos permite analizar y cuantificar la relación entre distintas variables económicas o financieras. En particular, se analiza el *efecto que una variación de una variable económica puede tener en la variación de otra variable*.

Algunas aplicaciones:

1. `Macroeconomía` Predicción de variables tales como el PIB, inflación o cuantificar la relación entre tasas de interés-inflación.
2. `Microeconomia` Cuantificar la relación entre educación y salario, factores productivos y producción, gastos en I+D y beneficios de la empresas.
3. `Finanzas` Análisis de volatilidad de los activos, modelos de valoración de activos, etc.  

<div style="border: 1px solid #007acc; background-color: #e6f2ff; padding: 10px; border-radius: 5px;">
   <strong> 📌 Idea Intuitiva  :</strong>  tenemos un fenómeno que está siendo influenciado por otras variables, por lo tanto, creemos que existe algún tipo de relación. Esta creencia que tenemos, debemos plasmarla en algo material, por lo tanto, necesitamos precisarla matemáticamente. Es decir, buscamos saber con la mayor precision posible, cómo funciona esta relación, por otro lado, ¿esta relación, es positiva?, ¿negativa?, ¿es fuerte?, ¿débil?
</div>

## 🦊 Supuestos de Gauss-Markov


## 🦖 Regresión Lineal

Intentamos explicar el comportamiento de cierto fenómeno a través de los siguientes componentes:

* **`variable explicada`** variable explicativa, independiente o a veces llamada regresora.
* **`variable dependiente`** variable dependiente, explicada o regresando.
* **`error`** perturbación aleatoria, error de estimación, shock exógeno.

En el caso de explicar a través de una sola variable $X_1$, tenemos:

\begin{equation*}
Y = f(X_1) + \epsilon
\end{equation*}

En el caso con más variables explicativas $k>1$, 

\begin{equation*}
Y = f(X_1, X_2, X_3, \cdots, X_k) + \epsilon
\end{equation*}

Cuando hablemos de *regresión simple*, estamos hablando de modelos con una sola variable explicativa, es decir:

\begin{equation*}
y = \beta_0 + \beta_1 x_1 + \epsilon
\end{equation*}

La interpretación del modelo anterior, es que la variable $y$ (variable observada) estaría siendo explicada por una constane $\beta_0$, una variable $x_1$ con un coeficiente $\beta_1$ y un error
$\epsilon$.

Por otro lado, cuando hablemos de  *regresión múltiple*, estaremos hablando de modelos con múltiples variables explicativas

\begin{equation*}
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 +  \beta_3 x_3 + \cdots  \beta_k x_k + \epsilon
\end{equation*}

La diferencia aquí, es que no solo consideramos $x_1$ para explicar $y$, sino que a todas las 
demás variables explicativas las cuales, cada una de ellas tiene su respectivo coeficiente $\beta_i$

**¿ Qué significan todas estas nomenclaturas ?** 🤔

* $\bf{y}$ es la variable de nuestro interes y se denomina variable explicada o dependiente.
  
* $\bf{x_1,x_2,\cdots,x_k}$ representan a las variables que consideramos relevantes para el análisis de la variable de nuestro interés, y se les denomina variables explicativas. El valor 
de $k$ indica el número de variables explicativas que estamos incluyendo en el modelo. 

* $\bf{\beta_1,\beta_2,\cdots,\beta_k}$ Son constantes que desconocemos su valor, pero que nos gustaría conocer ya que miden el *efecto que las variables regresoras tienen en la variable regresada*. Se les denomina coeficientes del modelo de regresión.

* $\bf{\epsilon}$ 
  * Teoricamente hablando ...
    * Variable aleatoria que perturba una supuesta relación estable entre $y$ y sus variables explicativas $x$, por eso es comunmente llamada perturbacion. 
    * Representa el efecto neto (positivo / negativo) sobre $y$ de todas las infinitas variables que no podemos captar con el modelo. 
  * Empiricamente hablando ...
    * Capta el error en la estimación de los parámetros, el error en la apropiada selección y número de variables explicativas, etc. En pocas palabras, no existe un modelo que explique de manera perfecta el comportamiento de una variable, por lo tanto, en la práctica, asumimos un cierto margen de error $\epsilon$

### Regresión Lineal Simple

**¿Cuál es el objetivo? 😴** 

1. Seleccionar las variables que impactan en $y$ (apoyados en teoría económica, financiera, etc).
2. Encontrar de qué manera y en qué magnitud, impactan estas variables (estimacion de $\beta_i$)
3. Comprobar si el modelo definido es robusto y realmente existe causalidad entre $y$ y las variables que creemos que explican su comportamiento.

En un modelo de regresión lineal podemos observar los valores de la variable dependiente y de las variables independientes. Desconocemos los valores de los coeficientes $\beta_i$ y no podemos observar el término del error $\epsilon$.

El principal objetivo es conocer los valores de los coeficientes, esto, es porque ellos *miden el efecto* que un cambio en su correspondiente variable regresora causa en la variable regresada.

**¿Cómo funciona el modelo?** ☕

Primero, consideremos la tasa de interés del tesoro americano a 10 años (Código FRED: `GS10`) que es la tasa de interés nominal anual que pagan los bonos del tesoro de EE.UU con vencimiento a 10 años. Se considera una tasa libre de riesgo a largo plazo. Por otro lado, consideremos a un ETF de bonos del Tesoro de largo plazo (Ticker Yahoo: `TLT`) que es es un fondo cotizado (ETF) que invierte exclusivamente en bonos del Tesoro de EE.UU. con vencimientos mayores a 20 años. 

<div style="display: flex; justify-content: space-around;">
  <img src="scatterplot.png" alt="Imagen 1" width="300">
  <img src="scatterplot2.png" alt="Imagen 2" width="300">
</div>

Cuando la tasa de interés sube (GS10), los bonos existentes en el mercado (como los que tiene TLT) pagan un cupón fijo que ahora es menos atractivo que las nuevas tasas del mercado. Por tanto, para igualar el atractivo, su precio de mercado baja. Cuando baj la tasa de interés (GS10), los bonos existentes, con cupones más altos que los nuevos bonos, se vuelven más valiosos, por tanto el precio del bono sube, y con ello sube TLT (relación lineal negativa).

**Idea 📌** Minimizar el error entre la observación $y_i$ y lo que se ajusta $\hat{y}$, es decir, debemos construir una forma en que, la distancia entre lo que se observa y lo que se ajusta, sea lo mas mínimo o pequeño posible (minimizar $\epsilon_i = y_i - \hat{y}$). Sin embargo, debido a que no existen distancias negativas, debemos ser capaces de construir una forma de que las distancias sean mayores que 0, es decir

\begin{equation*}
\epsilon_{i}^2 = (y_i - \hat{y})^2
\end{equation*}

Cabe destacar que la expresión anterior, es para un punto específico del diagrama de dispersión, sin embargo, debemos generalizarlo para todos los puntos del diagrama, por ende, debemos cuantificar el error de todos los puntos.

\begin{equation*}
\sum_{i=1}^{n}\epsilon_{i}^2 = \sum_{i=1}^{n}(y_i - \hat{y})^2
\end{equation*}


Por tanto, tenemos un modelo teorico definido como $y_i = \beta_0 + \beta_1 x_i + \epsilon$ y tenemos un modelo *ajustado* $\hat{y} = \widehat{\beta_0} + \widehat{\beta_1} x_i$, por lo tanto, la funcion a minimizar es: 

\begin{equation*}
S := \sum_{i=1}^{n}\epsilon_{i}^2 = \sum_{i=1}^{n}(y_i -  \widehat{\beta_0} - \widehat{\beta_1} x_i)^2
\end{equation*}

### Métodos de Estimación

#### Mínimos cuadrados

Vamos a explicar el método para hallar las estimaciones $\widehat{\beta_0}$ y $\widehat{\beta_1}$. Dicho método se conoce como **mínimos cuadrados**.

Dada una observación cualquiera de la muestra, $(x_i,y_i)$, podremos separar la componente $y_i$ como la suma de su valor predicho por el modelo y el error cometido: 

\begin{equation*}
y_i= \beta_0 + \beta_1 x_i + \epsilon_i \rightarrow \epsilon_i = y_i-(\beta_0+\beta_1x_i)
\end{equation*}

Llamamos **error cuadrático teórico** de este modelo a la suma al cuadrado de todos los errores cometidos por los valores de la muestra: 

\begin{equation*}
SS_E=\sum_{i=1}^n\epsilon_i^2=\sum_{i=1}^n(y_i-(\beta_0+\beta_1x_i))^2
\end{equation*}

La regresión lineal por mínimos cuadrados consiste en hallar los estimadores $\widehat{\beta_0}$ y $\widehat{\beta_1}$ que minimicen el error cuadrático teórico. Es sencillo demostrar que los valores que optimizan son 

\begin{equation*}
\widehat{\beta_{1}} = \frac{\sum_{i=1}^{n}(x_i -\overline{x})(y_i - \overline{y})}{\sum_{i=1}^n (x_i - \overline{x})^2} \quad \widehat{\beta_{0}} = \overline{y} - \widehat{\beta_1}\overline{x}
\end{equation*}

#### Máxima Verosimilitud

#### Test y Diagnósticos

## 🐙 Simulación  

## 🦕 Regresión Lineal Múltiple